Cellule 1 — Imports & config MLflow

In [1]:
# CELL 1 — Imports & configuration

import json
import yaml
import numpy as np
import pandas as pd
import mlflow
from mlflow import MlflowClient

from sklearn.model_selection import StratifiedKFold, cross_val_score

import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from app.features_optiweb import apply_eda

SEED = 42
np.random.seed(SEED)

EXPERIMENT_NAME = "optiweb_topk"
mlflow.set_experiment(EXPERIMENT_NAME)

MODEL_NAME = "optiweb_params"
MODEL_VERSION = 2   # <-- ta version optimisée

print("Experiment:", EXPERIMENT_NAME)
print("Using model:", MODEL_NAME, "version", MODEL_VERSION)


Experiment: optiweb_topk
Using model: optiweb_params version 2


Cellule 2 — Charger modèle + artifacts depuis MLflow Registry

In [2]:
# CELL 2 — Chargement du modèle & des artifacts depuis le Model Registry

client = MlflowClient()

mv = client.get_model_version(MODEL_NAME, str(MODEL_VERSION))
run_id = mv.run_id
print("ModelVersion run_id:", run_id)

model_uri = f"models:/{MODEL_NAME}/{MODEL_VERSION}"

# === Charger le modèle depuis MLflow (flavor sklearn) ===
from mlflow.sklearn import load_model as load_sklearn_model
base_model = load_sklearn_model(model_uri)

print("Loaded estimator type:", type(base_model))

# === Charger les artifacts ===
features_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path="features_used.json"
)
fi_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path="feature_importance.csv"
)
params_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path="best_params.yaml"
)

with open(features_path, "r") as f:
    features_payload = json.load(f)

if isinstance(features_payload, dict) and "features_used" in features_payload:
    features_used = features_payload["features_used"]
else:
    features_used = features_payload

fi = pd.read_csv(fi_path)
with open(params_path, "r") as f:
    best_params = yaml.safe_load(f)

fi = fi.sort_values("importance", ascending=False)
features_ranked = fi["feature"].tolist()

print("Nb features used:", len(features_used))
print("Top 10 features:", features_ranked[:10])


c:\Users\suean\OneDrive\Desktop\tom\OPCL2\P8\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelVersion run_id: 6afd1e84d8f449919dca1b36c3103286
Loaded estimator type: <class 'lightgbm.sklearn.LGBMClassifier'>


Nb features used: 765
Top 10 features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH', 'AMT_ANNUITY', 'APPROVED_CNT_PAYMENT_MEAN', 'DAYS_ID_PUBLISH', 'INSTAL_DPD_MEAN', 'AMT_CREDIT']


Cellule 3 — Recharger le dataset avec apply_eda + imbalance

In [3]:
# CELL 3 — Rechargement du dataset & alignement des features

# tu peux mettre nrows=None (full) ou une sous-sample temporaire pour aller plus vite
NROWS = None

X_train, y_train, X_test, test_ids = apply_eda(nrows=NROWS)
print("Raw X_train shape:", X_train.shape)

# On restreint X_train aux features utilisées par le modèle (et même ordre)
X_train = X_train[features_used]
print("Aligned X_train shape:", X_train.shape)

from collections import Counter
cnt = Counter(y_train)
pos = cnt[1]
neg = cnt[0]
scale_pos = neg / pos
print("Target distribution:", cnt, " -> scale_pos_weight =", scale_pos)

# CV commun
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)


Raw X_train shape: (307507, 765)
Aligned X_train shape: (307507, 765)
Target distribution: Counter({0: 282682, 1: 24825})  -> scale_pos_weight = 11.386988922457201


Cellule 4 — Helper build_model pour garder même famille / params

On va utiliser les mêmes hyperparams que le modèle du registry.


In [8]:
# CELLULE 4 — Reconstruction propre du modèle LightGBM venant du registry

# 1) Vérification modèle
if not isinstance(base_model, lgb.LGBMClassifier):
    raise TypeError(
        "Le modèle chargé n'est pas un LightGBMClassifier. "
        "Ce notebook top-K est conçu pour LightGBM uniquement."
    )

# 2) Récupération des hyperparams optimisés (Optuna)
params = base_model.get_params()

# 3) Mise à jour des paramètres critiques
params.update({
    "device_type": "gpu",
    "verbosity": -1,
    "random_state": SEED,
    "scale_pos_weight": scale_pos,   # recalculé dans la Cellule 3
})

# 4) Reconstruction propre d’un modèle NEUF (non fit)
def build_lgbm_clone():
    """Construit un clone LightGBM propre avec les bons params & GPU."""
    return lgb.LGBMClassifier(**params)

baseline_model = build_lgbm_clone()

print("\n=== Modèle LightGBM reconstruit ===")
print("Type:", type(baseline_model))
print("Params clés:")
print(" - n_estimators:", params["n_estimators"])
print(" - num_leaves:", params["num_leaves"])
print(" - learning_rate:", params["learning_rate"])
print(" - scale_pos_weight:", params["scale_pos_weight"])
print(" - device_type:", params["device_type"])
print("\nbuild_lgbm_clone() prêt pour le top-K !")



=== Modèle LightGBM reconstruit ===
Type: <class 'lightgbm.sklearn.LGBMClassifier'>
Params clés:
 - n_estimators: 875
 - num_leaves: 23
 - learning_rate: 0.02533714281950245
 - scale_pos_weight: 11.386988922457201
 - device_type: gpu

build_lgbm_clone() prêt pour le top-K !


Cellule 5 — Baseline AUC full features (référence)

In [9]:
# CELL 5 — AUC baseline du modèle optimisé avec toutes les features

scores_full = cross_val_score(
    baseline_model,
    X_train,
    y_train,
    cv=cv,
    scoring="roc_auc",
    n_jobs=1
)

baseline_auc = float(scores_full.mean())
baseline_std = float(scores_full.std())

print(f"Baseline (full features) AUC = {baseline_auc:.4f} ± {baseline_std:.4f}")


Baseline (full features) AUC = 0.7871 ± 0.0039


Cellule 6 — Expérience top-K (réutilise le modèle du registry comme gabarit)

In [11]:
# CELL 6 — Sweep sur K (top-K features) avec MLflow + PRINTS lisibles

K_GRID = [30, 60, 100, 300, len(features_ranked)]

results_topk = []

print("\n==============================================")
print("     🔎 DÉBUT DU TOP-K SWEEP (LightGBM) ")
print("==============================================\n")
print(f"AUC baseline (full model) = {baseline_auc:.5f} ± {baseline_std:.5f}\n")


with mlflow.start_run(run_name="topk_sweep_from_registry") as parent_run:

    mlflow.log_param("source_model_name", MODEL_NAME)
    mlflow.log_param("source_model_version", MODEL_VERSION)
    mlflow.log_metric("baseline_auc_full", baseline_auc)
    mlflow.log_metric("baseline_auc_std_full", baseline_std)

    for K in K_GRID:

        topk_features = features_ranked[:K]
        Xk = X_train[topk_features]

        print(f"\n=== 🔵 K = {K} features ===")
        print("Top features:", topk_features[:5], "...")

        with mlflow.start_run(run_name=f"topK_{K}", nested=True):

            model_k = build_lgbm_clone()

            scores_k = cross_val_score(
                model_k,
                Xk,
                y_train,
                cv=cv,
                scoring="roc_auc",
                n_jobs=1
            )

            mean_k = float(scores_k.mean())
            std_k  = float(scores_k.std())
            delta  = mean_k - baseline_auc

            # MLflow logs
            mlflow.log_param("K", K)
            mlflow.log_metric("roc_auc_mean", mean_k)
            mlflow.log_metric("roc_auc_std", std_k)
            mlflow.log_metric("delta_auc_vs_full", delta)

            with open("topk_features.txt", "w") as f:
                f.write("\n".join(topk_features))
            mlflow.log_artifact("topk_features.txt")

            # Ajout au tableau final
            results_topk.append(
                {"K": K, "auc": mean_k, "std": std_k, "delta": delta}
            )

            # PRINTS LISIBLE
            print(f"AUC = {mean_k:.5f} ± {std_k:.5f}   Δ vs full = {delta:+.5f}")


# Résultat final sous forme de tableau
results_topk_df = pd.DataFrame(results_topk).sort_values("K")

print("\n\n==============================================")
print(" 🔥 Résultats finaux du TOP-K sweep")
print("==============================================\n")
print(results_topk_df.to_string(index=False))

results_topk_df



     🔎 DÉBUT DU TOP-K SWEEP (LightGBM) 

AUC baseline (full model) = 0.78715 ± 0.00393


=== 🔵 K = 30 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.77467 ± 0.00371   Δ vs full = -0.01248

=== 🔵 K = 60 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.78350 ± 0.00378   Δ vs full = -0.00365

=== 🔵 K = 100 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.78662 ± 0.00395   Δ vs full = -0.00053

=== 🔵 K = 300 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.78742 ± 0.00388   Δ vs full = +0.00027

=== 🔵 K = 765 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.78706 ± 0.00377   Δ vs full = -0.00009


 🔥 Résultats finaux du TOP-K sweep

  K      auc   

,K,auc,std,delta
0,30,0.774669,0.003711,-0.012480
1,60,0.783498,0.003784,-0.003651
2,100,0.786615,0.003955,-0.000534
3,300,0.787418,0.003876,0.000269
4,765,0.787060,0.003767,-0.000089


Au vue de ses résultats, je vais affiner un peu plus avec tres peu de features !

In [12]:
# CELL 6 — Sweep sur K (top-K features) avec MLflow + PRINTS lisibles

K_GRID = [5, 15, 20, 40]

results_topk = []

print("\n==============================================")
print("     🔎 DÉBUT DU TOP-K SWEEP (LightGBM) ")
print("==============================================\n")
print(f"AUC baseline (full model) = {baseline_auc:.5f} ± {baseline_std:.5f}\n")


with mlflow.start_run(run_name="topk_sweep_from_registry") as parent_run:

    mlflow.log_param("source_model_name", MODEL_NAME)
    mlflow.log_param("source_model_version", MODEL_VERSION)
    mlflow.log_metric("baseline_auc_full", baseline_auc)
    mlflow.log_metric("baseline_auc_std_full", baseline_std)

    for K in K_GRID:

        topk_features = features_ranked[:K]
        Xk = X_train[topk_features]

        print(f"\n=== 🔵 K = {K} features ===")
        print("Top features:", topk_features[:5], "...")

        with mlflow.start_run(run_name=f"topK_{K}", nested=True):

            model_k = build_lgbm_clone()

            scores_k = cross_val_score(
                model_k,
                Xk,
                y_train,
                cv=cv,
                scoring="roc_auc",
                n_jobs=1
            )

            mean_k = float(scores_k.mean())
            std_k  = float(scores_k.std())
            delta  = mean_k - baseline_auc

            # MLflow logs
            mlflow.log_param("K", K)
            mlflow.log_metric("roc_auc_mean", mean_k)
            mlflow.log_metric("roc_auc_std", std_k)
            mlflow.log_metric("delta_auc_vs_full", delta)

            with open("topk_features.txt", "w") as f:
                f.write("\n".join(topk_features))
            mlflow.log_artifact("topk_features.txt")

            # Ajout au tableau final
            results_topk.append(
                {"K": K, "auc": mean_k, "std": std_k, "delta": delta}
            )

            # PRINTS LISIBLE
            print(f"AUC = {mean_k:.5f} ± {std_k:.5f}   Δ vs full = {delta:+.5f}")


# Résultat final sous forme de tableau
results_topk_df = pd.DataFrame(results_topk).sort_values("K")

print("\n\n==============================================")
print(" 🔥 Résultats finaux du TOP-K sweep")
print("==============================================\n")
print(results_topk_df.to_string(index=False))

results_topk_df



     🔎 DÉBUT DU TOP-K SWEEP (LightGBM) 

AUC baseline (full model) = 0.78715 ± 0.00393


=== 🔵 K = 5 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.75230 ± 0.00478   Δ vs full = -0.03485

=== 🔵 K = 15 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.76565 ± 0.00341   Δ vs full = -0.02150

=== 🔵 K = 20 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.77045 ± 0.00419   Δ vs full = -0.01669

=== 🔵 K = 40 features ===
Top features: ['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'DAYS_BIRTH'] ...
AUC = 0.78062 ± 0.00426   Δ vs full = -0.00653


 🔥 Résultats finaux du TOP-K sweep

 K      auc      std     delta
 5 0.752300 0.004776 -0.034849
15 0.765653 0.003414 -0.021496
20 0.770454 0.004186 -0.016695
40 0.780620 0.004258 -0.006529


,K,auc,std,delta
0,5,0.752300,0.004776,-0.034849
1,15,0.765653,0.003414,-0.021496
2,20,0.770454,0.004186,-0.016695
3,40,0.780620,0.004258,-0.006529


OK ! on va save le top 20 et top 40 as light et med, la version full sera dev si le projet est vendu ✌️🤣

CELLULE 8 — Export des modèles Top-20 & Top-40 dans le registry

In [13]:
# ================================================
# CELLULE 8 — Enregistrer optiweb_top20 et optiweb_top40
# ================================================

from mlflow import MlflowClient

client = MlflowClient()

def save_topk_model(K, model_name):
    """
    Entraîne le modèle avec top-K features,
    loggue les artefacts, et publie dans le Model Registry.
    """

    print(f"\n=== 🚀 Publication modèle {model_name} (K={K}) ===")

    # --- 1) Select top-K features
    topk_features = features_ranked[:K]
    Xk = X_train[topk_features]

    # --- 2) Rebuild LightGBM
    model_k = build_lgbm_clone()
    model_k.fit(Xk, y_train)

    # --- 3) Create feature_importance.csv for this sub-model
    fi_k = pd.DataFrame({
        "feature": topk_features,
        "importance": model_k.feature_importances_
    })
    fi_k.to_csv("feature_importance.csv", index=False)

    # --- 4) Save features_used.json
    with open("features_used.json", "w") as f:
        json.dump({"features_used": topk_features}, f, indent=2)

    # --- 5) Save params.yaml (same Optuna hyperparams)
    with open("best_params.yaml", "w") as f:
        yaml.dump(best_params, f)

    # --- 6) MLflow logging
    with mlflow.start_run(run_name=f"export_{model_name}") as run:

        # Log artifacts
        mlflow.log_artifact("features_used.json")
        mlflow.log_artifact("feature_importance.csv")
        mlflow.log_artifact("best_params.yaml")

        # Log model
        model_info = mlflow.lightgbm.log_model(
            lgb_model=model_k,
            name=model_name,
            input_example=Xk.iloc[:5]
        )

        run_id = run.info.run_id

    # --- 7) Register in MLflow Model Registry
    try:
        client.create_registered_model(model_name)
    except:
        pass  # Already exists

    mv = client.create_model_version(
        name=model_name,
        source=model_info.model_uri.replace("runs:/", "mlflow-artifacts:/"),
        run_id=run_id
    )

    client.transition_model_version_stage(
        name=model_name,
        version=mv.version,
        stage="Staging"
    )

    print(f"🎉 Modèle {model_name} publié — version {mv.version}")


# ----- Enregistrement Top-20 -----
save_topk_model(20, "optiweb_top20")

# ----- Enregistrement Top-40 -----
save_topk_model(40, "optiweb_top40")



=== 🚀 Publication modèle optiweb_top20 (K=20) ===


2025/11/13 16:26:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\suean\OneDrive\Desktop\tom\OPCL2\P8\.venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\suean\OneDrive\Desktop\tom\OPCL2\P8\.venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hin

🎉 Modèle optiweb_top20 publié — version 1

=== 🚀 Publication modèle optiweb_top40 (K=40) ===


2025/11/13 16:26:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\suean\OneDrive\Desktop\tom\OPCL2\P8\.venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\suean\OneDrive\Desktop\tom\OPCL2\P8\.venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hin

🎉 Modèle optiweb_top40 publié — version 1



C:\Users\suean\AppData\Local\Temp\ipykernel_27100\1626055401.py:69: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
